In [2]:
from transformers import AutoConfig, AutoModelForSeq2SeqLM, AutoTokenizer

/home/ubuntu/miniconda3/envs/rebel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
config = AutoConfig.from_pretrained('/home/ubuntu/rebel/model/pretrained-mt5-base')
tokenizer = AutoTokenizer.from_pretrained('/home/ubuntu/rebel/model/pretrained-mt5-base')
model = AutoModelForSeq2SeqLM.from_pretrained('/home/ubuntu/rebel/model/pretrained-mt5-base')

In [13]:
import torch

def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = (
        text
        .replace("<obj>", " <obj> ")
        .replace("<subj>", " <subj> ")
        .replace("<triplet>", " <triplet> ")
        .replace("<s>", "")
        .replace("</s>", "")
        .replace("<pad>", "")
        .strip()
    )
    for i in range(100):
        text = text.replace(f"<extra_id_{i}>", "")
    current = 'x'
    for token in text.split():
        token = token.strip()
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject.strip() and relation.strip() and object_.strip():
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    triplets = [item for item in triplets if item["head"] and item["type"] and item["tail"]]
    return triplets


def _predict(model, tokenizer, text):
    gen_kwargs = {
        "max_length": 128,
        "early_stopping": False,
        "length_penalty": 0,
        "no_repeat_ngram_size": 0,
        "num_beams": 3,
    }
    
    model_inputs = tokenizer(text, max_length=256, padding=True, truncation=True, return_tensors = 'pt')
    batch_size = model_inputs['input_ids'].shape[0]
    decoder_inputs = torch.tensor([[0, 250100] for _ in range(batch_size)])
    generated_tokens = model.generate(
        model_inputs["input_ids"].to(model.device),
        attention_mask=model_inputs["attention_mask"].to(model.device),
        decoder_input_ids=decoder_inputs.to(model.device),
        **gen_kwargs,
    )
    
    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=False)
    return decoded_preds

predict = lambda text: _predict(model, tokenizer, text)

In [14]:
text = """
اینشتین در ۱۴ مارس ۱۸۷۹ در اولم آلمان به دنیا آمد. خانواده‌اش یهودی بودند و در ۱۸۸۰ به مونیخ مهاجرت کردند. پدرش هرمان یک فروشنده و مهندس بود و عموی او یاکوب با او تجارت کوچکی برای نصب گاز و آب تأسیس کردند. این تجارت موفق بود و در ۱۸۸۵ کارخانه تولید لوازم الکتریکی راه‌اندازی کردند.

اینشتین در ۵ سالگی به مدرسه ابتدایی کاتولیک در مونیخ رفت و سه سال بعد به مدرسه لوتیپولد فرستاده شد. او در ۱۴ سالگی به ایتالیا رفت و در پاویا ساکن شد. پدرش تمایل داشت که او رشته مهندسی برق را ادامه دهد اما اینشتین به‌خاطر برخوردهایی که با مسئولان مدرسه داشت از آن‌ها متنفر شده بود.

اینشتین حتی از دوران جوانی تبحر خاصی در ریاضی و فیزیک داشت. او از ۱۲ سالگی شروع به خودآموزی جبر و هندسه اقلیدسی کرد و در سن ۱۲ سالگی اثباتی اختصاصی برای قضیه فیثاغورس بیابد. او در سن ۱۴ سالگی به گفته خودش در انتگرال و حساب دیفرانسیل به استادی رسیده بود.

اینشتین در ۱۸۹۴ کارخانه پدرش را ترک کرد و به ایتالیا رفت. او در پاویا ساکن شد و در ۱۸۹۵ به دانشگاه لا ساپیenza در رم رفت. او در آنجا با ماکس پلانک آشنا شد و تحت تأثیر او قرار گرفت.

"""
sentences = [(x.strip() + ".") for item in text.strip().split("\n") for x in item.strip().split(".") if x.strip()]

In [15]:
for sen in sentences:
    decoded_preds = predict(sen.strip())
    print(f"input: {sen}, output: {extract_triplets(decoded_preds[0])[0]}")

input: اینشتین در ۱۴ مارس ۱۸۷۹ در اولم آلمان به دنیا آمد., output: {'head': 'اولم', 'type': 'کشور', 'tail': 'آلمان'}
input: خانواده‌اش یهودی بودند و در ۱۸۸۰ به مونیخ مهاجرت کردند., output: {'head': 'مونیخ', 'type': 'کشور', 'tail': 'یهودی'}
input: پدرش هرمان یک فروشنده و مهندس بود و عموی او یاکوب با او تجارت کوچکی برای نصب گاز و آب تأسیس کردند., output: {'head': 'هرمان', 'type': 'همنیا', 'tail': 'یاکوب'}
input: این تجارت موفق بود و در ۱۸۸۵ کارخانه تولید لوازم الکتریکی راه‌اندازی کردند., output: {'head': 'لوازم الکتریکی', 'type': 'زیرردۀ', 'tail': 'لوازم الکتریکی'}
input: اینشتین در ۵ سالگی به مدرسه ابتدایی کاتولیک در مونیخ رفت و سه سال بعد به مدرسه لوتیپولد فرستاده شد., output: {'head': 'مدرسه ابتدایی کاتولیک', 'type': 'موقعیت در تقسیمات کشوری', 'tail': 'مونیخ'}
input: او در ۱۴ سالگی به ایتالیا رفت و در پاویا ساکن شد., output: {'head': 'پاویا', 'type': 'کشور', 'tail': 'ایتالیا'}
input: پدرش تمایل داشت که او رشته مهندسی برق را ادامه دهد اما اینشتین به‌خاطر برخوردهایی که با مسئولان مدرسه 